# Deutsch-Josza Implementation

In [42]:
from qiskit import QuantumCircuit

In [127]:
import random

class Oracle(object):
    def __init__(self, n):
        self.n = n
    
    def build(self):
        print(f"Building oracle: {self.__class__.__name__} {self.__dict__}")
        circuit = self._build(self.n)
        print(circuit.draw())
        return circuit
        
    def _build(self, n):
        raise NotImplemented()

    
class ConstantOracle(Oracle):
    def __init__(self, is_zero, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.is_zero = is_zero
    
    def _build(self, n):
        circuit = QuantumCircuit(n+1, name=self.__class__.__name__)
        if not self.is_zero:
            circuit.x(n)
            
        return circuit
    
class BalancedOracle(Oracle):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        
    def _build(self, n):
        circuit = QuantumCircuit(n+1)
        for i in range(n):
            circuit.cx(i, n)
            
        return circuit

def get_random_oracle(n):
    return random.choice((ConstantOracle(False, n), ConstantOracle(True, n), BalancedOracle(n))).build()

In [161]:
n =10
oracle = get_random_oracle(n)
circuit = QuantumCircuit(n+1, n)

circuit.x(n)
for i in range(n+1):
    circuit.h(i)

circuit.append(oracle.to_instruction(), list(range(n+1)))

for i in range(n):
    circuit.h(i)
    circuit.measure([i], [i])
circuit.draw()

Building oracle: ConstantOracle {'n': 10, 'is_zero': True}
      
 q_0: 
      
 q_1: 
      
 q_2: 
      
 q_3: 
      
 q_4: 
      
 q_5: 
      
 q_6: 
      
 q_7: 
      
 q_8: 
      
 q_9: 
      
q_10: 
      


┌───┐     ┌──────────────────┐┌───┐┌─┐                           
 q_0: ┤ H ├─────┤0                 ├┤ H ├┤M├───────────────────────────
      ├───┤     │                  │├───┤└╥┘┌─┐                        
 q_1: ┤ H ├─────┤1                 ├┤ H ├─╫─┤M├────────────────────────
      ├───┤     │                  │├───┤ ║ └╥┘┌─┐                     
 q_2: ┤ H ├─────┤2                 ├┤ H ├─╫──╫─┤M├─────────────────────
      ├───┤     │                  │├───┤ ║  ║ └╥┘┌─┐                  
 q_3: ┤ H ├─────┤3                 ├┤ H ├─╫──╫──╫─┤M├──────────────────
      ├───┤     │                  │├───┤ ║  ║  ║ └╥┘┌─┐               
 q_4: ┤ H ├─────┤4                 ├┤ H ├─╫──╫──╫──╫─┤M├───────────────
      ├───┤     │                  │├───┤ ║  ║  ║  ║ └╥┘┌─┐            
 q_5: ┤ H ├─────┤5  ConstantOracle ├┤ H ├─╫──╫──╫──╫──╫─┤M├────────────
      ├───┤     │                  │├───┤ ║  ║  ║  ║  ║ └╥┘┌─┐         
 q_6: ┤ H ├─────┤6                 ├┤ H ├─╫──╫──╫──╫──╫──╫─┤M├─────────
      ├───┤     │                  │├───┤ ║  ║  ║  ║  ║  ║ └╥┘┌─┐      
 q_7: ┤ H ├─────┤7                 ├┤ H ├─╫──╫──╫──╫──╫──╫──╫─┤M├──────
      ├───┤     │                  │├───┤ ║  ║  ║  ║  ║  ║  ║ └╥┘┌─┐   
 q_8: ┤ H ├─────┤8                 ├┤ H ├─╫──╫──╫──╫──╫──╫──╫──╫─┤M├───
      ├───┤     │                  │├───┤ ║  ║  ║  ║  ║  ║  ║  ║ └╥┘┌─┐
 q_9: ┤ H ├─────┤9                 ├┤ H ├─╫──╫──╫──╫──╫──╫──╫──╫──╫─┤M├
      ├───┤┌───┐│                  │└───┘ ║  ║  ║  ║  ║  ║  ║  ║  ║ └╥┘
q_10: ┤ X ├┤ H ├┤10                ├──────╫──╫──╫──╫──╫──╫──╫──╫──╫──╫─
      └───┘└───┘└──────────────────┘      ║  ║  ║  ║  ║  ║  ║  ║  ║  ║ 
c: 10/════════════════════════════════════╩══╩══╩══╩══╩══╩══╩══╩══╩══╩═
                                          0  1  2  3  4  5  6  7  8  9

In [135]:
from qiskit import transpile
from qiskit_aer import AerSimulator

In [162]:
simulator = AerSimulator()
job = simulator.run(transpile(circuit, simulator), shots=1)
result = job.result()
print(list(result.get_counts())[0])

0000000000
